In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install ultralytics --upgrade -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is in

In [2]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
path = 'MART-NISAN 2024 TARIHLI TURKIYE GENELINDE MOBESE TRAFIK KAZA GORUNTULERI.mp4'

In [12]:
model = YOLO("best.pt")

In [17]:
results = model(source=path, stream=True, conf=0.3)


In [25]:
import cv2
from collections import deque
cap   = cv2.VideoCapture(path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out   = cv2.VideoWriter('output_with_score.mp4', fourcc, 30,
                        (int(cap.get(3)), int(cap.get(4))))


In [26]:
vehicle_weights = {
    'bicycle':   0.5,
    'bus':       2.0,
    'car':       1.0,
    'motorbike': 1.0
}

# smoothing buffers
veh_score_hist = deque(maxlen=5)
ped_count_hist = deque(maxlen=5)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True, conf=0.1)
    r = next(results)

    veh_counts = {c:0 for c in vehicle_weights}
    ped_count = 0

    for cls in r.boxes.cls:
        name = model.names[int(cls.item())]
        if name in veh_counts:
            veh_counts[name] += 1
        elif name == 'person':
            ped_count += 1

    veh_score = sum(veh_counts[c] * vehicle_weights[c] for c in veh_counts)

    # smooth both metrics
    veh_score_hist.append(veh_score)
    ped_count_hist.append(ped_count)
    smooth_veh = sum(veh_score_hist) / len(veh_score_hist)
    smooth_ped = sum(ped_count_hist) / len(ped_count_hist)

    frame = r.plot()

    # overlay both metrics
    cv2.putText(frame, f'Congestion: {smooth_veh:.1f}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)
    cv2.putText(frame, f'Pedestrians: {int(smooth_ped)}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 2)

    out.write(frame)

cap.release()
out.release()
print("Done! Video saved as output_with_score.mp4")